# Common imports & library functions

In [1]:
import doctest

# Day 1: Calorie Counting

In [17]:
def parse_calories(calories_txt):
  r"""
  >>> parse_calories("1\n2\n3\n\n4\n5")
  [[1, 2, 3], [4, 5]]
  """
  return [[int(c) for c in items_per_elf.split('\n')]
          for items_per_elf in calories_txt.split('\n\n')]

def max_calories(calories, n=1):
  """
  >>> max_calories([[1, 2, 3], [4, 5]], n=1)
  9
  >>> max_calories([[1, 2, 3], [4, 5]], n=2)
  [9, 6]
  >>> max_calories([[1000, 2000, 3000], [4000], [5000, 6000], [7000, 8000, 9000], [10000]])
  24000
  """
  sums = sum(cs) for cs in calories
  if n == 1:
    return max(sums)
  else:
    return sorted(sums, reverse=True)[:n]

In [18]:
doctest.run_docstring_examples(parse_calories, globs=None, verbose=True)
doctest.run_docstring_examples(max_calories, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_calories("1\n2\n3\n\n4\n5")
Expecting:
    [[1, 2, 3], [4, 5]]
ok
Finding tests in NoName
Trying:
    max_calories([[1, 2, 3], [4, 5]], n=1)
Expecting:
    9
**********************************************************************
File "__main__", line 11, in NoName
Failed example:
    max_calories([[1, 2, 3], [4, 5]], n=1)
Expected:
    9
Got:
    [9]
Trying:
    max_calories([[1, 2, 3], [4, 5]], n=2)
Expecting:
    [9, 6]
ok
Trying:
    max_calories([[1000, 2000, 3000], [4000], [5000, 6000], [7000, 8000, 9000], [10000]])
Expecting:
    24000
**********************************************************************
File "__main__", line 15, in NoName
Failed example:
    max_calories([[1000, 2000, 3000], [4000], [5000, 6000], [7000, 8000, 9000], [10000]])
Expected:
    24000
Got:
    [24000]


In [14]:
# Final answers
with open('day1.txt') as f:
    calories = parse_calories(f.read().strip())
    print('Part 1: ', max_calories(calories))

Part 1:  71506
